In [1]:
import pandas as pd
import requests
from lxml import etree
import time

In [2]:
ids_to_scrape = pd.read_csv("../data/custom/no_rat_id.csv")

In [19]:
### Gathering all Ratings for a single Game
def get_rating(game_id = 391565):
    soup_url = f"https://api.geekdo.com/xmlapi2/thing?id={game_id}&ratingcomments=1"

    response = requests.get(soup_url)
    if response.status_code != 200:                         # Handle Errors
        if response.status_code == 429:                     # Handle (To many Requests) - Error
            a=.5                                            # Define Sleeptime
            while response.status_code == 429:
                time.sleep(a)                               # Pause Execution for a seconds
                a =a+.1                                     # Prolong Sleep-Time
                response = requests.get(soup_url)
                if (response.status_code != 200) & (response.status_code != 429):   # Handle other Errors
                    raise Exception(f"Error retrieving data. Status code: {response.status_code} | While looking for game with id {game_id}")
        else:                                               # Handle other Errors
            raise Exception(f"Error retrieving data. Status code: {response.status_code} | While looking for game with id {game_id}")

    xml_content = response.content
    xml_tree = etree.fromstring(xml_content)                # Preparing Response for Feature-Extraction

    comments = xml_tree.xpath("//comment")                  # Selecting Features
    comments_dict = {}                                      # Creating empty Results-Dictionary

    for comment in comments:                                        # Looping through all Ratings
        username = comment.get("username")                          # Getting Username
        rating = comment.get("rating")                              # Getting Rating
        comments_dict.setdefault("Username", []).append(username)   # Adding Username to Dictionary
        comments_dict.setdefault("Rating", []).append(rating)       # Adding Rating to Dictionary   
        comments_dict.setdefault("BGGId", []).append(game_id)       # Adding BGGId to Dictionary


    df = pd.DataFrame(comments_dict)                        # Converting Dictionary into Pandas-DataFrame
    return(df)                                              # Returning Dataframe


In [20]:
a=0                                                                                                 # Initiate Counter
Rating_frame = pd.DataFrame(columns=['Username', 'Rating','BGGId'])                                 # Preparing Rating-DataFrame

for ID_game in ids_to_scrape['bgg_id']:                                                             # Initiating Loop
    a+=1                                                                                            # Counting Loops
    Rating_frame = pd.concat([Rating_frame, get_rating(game_id = ID_game)], ignore_index=True)      # Adding aquired Ratings to Rating-DataFrame
    if a == 20:                                                                                     # Condition for Backup-Creation
        a = 0                                                                                       # Resetting Counter
        Rating_frame.to_csv("../data/custom/extra_ratings.csv", index=False)                        # Creating a local Backup of the Rating-DataFrame